# Example 8: Signal to Noise and PSF

Máster en Astrofísica UCM
Técnicas Experimentales en Astrofísica

Jaime Zamorano, Nicolás Cardiel and Sergio Pascual

v1 2021/02/04

In [ ]:
import matplotlib

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.patches import Rectangle
from mpl_toolkits.axes_grid1 import make_axes_locatable

from astropy.io import fits
from astropy.stats import sigma_clip, sigma_clipped_stats, mad_std

import ccdproc
from ccdproc import CCDData, Combiner
from ccdproc import median_filter

import numpy as np
#import numpy.ma as ma

from photutils import DAOStarFinder
from photutils import find_peaks
from photutils import CircularAperture
from photutils import CircularAnnulus
from photutils import aperture_photometry

In [ ]:
plt.style.use('./tea.mplstyle')   # Some parameters for nicer graphs

Reading and displaying the image data and a region to be used in the first part of the example

In [ ]:
target_url ="https://guaix.fis.ucm.es/~jaz/master_TEA/FITS_files/ucmP_0050.fits"
target_url ="ftp://astrax.fis.ucm.es/pub/users/jaz/NOT_2008_04_12-14/N2/AL13055.fits"

In [ ]:
image = fits.getdata(target_url)
print(fits.info(target_url))
header = fits.getheader(target_url)
#print(header)

In [ ]:
#x1, x2, y1, y2 = 100, 500, 300, 700
x1, x2, y1, y2 = 800, 1300, 300, 800
image_1 = image[x1:x2, y1:y2]
image_2 = image_1[250:300, 110:160]
vmin,vmax = 1200,10000
fig, axarr = plt.subplots(ncols=3, nrows=1, figsize=(14, 9))
axarr[0].imshow(image, cmap='gray', vmin=vmin, vmax=vmax,norm=LogNorm())
axarr[1].imshow(image_1, cmap='gray', vmin=vmin, vmax=vmax,norm=LogNorm())
axarr[2].imshow(image_2, cmap='gray', vmin=vmin, vmax=vmax,norm=LogNorm())

In [ ]:
vmin,vmax = 5000,24000
fig, ax = plt.subplots(figsize=(6, 6))
img = ax.imshow(image_2, cmap='viridis',vmin=vmin, vmax=vmax,norm=LogNorm())
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(img, cax=cax)
ax.grid()
print(np.max(image_2))

In [ ]:
# auxiliary function to display a rectangle and compute mean value within it
def draw_rectangle(ax, image_data, x1, x2, y1, y2, color, text=False):
    ax.plot((x1, x1), (y1, y2), color, lw=1)
    ax.plot((x2, x2), (y1, y2), color, lw=1)
    ax.plot((x1, x2), (y1, y1), color, lw=1)
    ax.plot((x1, x2), (y2, y2), color, lw=1)
    if text:
        media = image_data[y1:y2,x1:x2].mean()
        std   = image_data[y1:y2,x1:x2].std()
        ax.text((x1+x2)/2, y1+(y2-y1)/8, str(int(media)), 
                ha='center', va='center', color=color, fontsize=15)        
        ax.text((x1+x2)/2, y2-(y2-y1)/8, str(round(std,1)), 
                ha='center', va='top', color=color, fontsize=15)
    return media, std

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 10))
img = ax.imshow(image_2, cmap='viridis', vmin=1000, vmax=6000)
ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(img, cax=cax, label='Number of counts')

x1, x2, y1, y2 = 0,50,20,25
draw_rectangle(ax,image_2,x1, x2, y1, y2,color='r')

In [ ]:
box_1 = image_2[y1:y2,:]
print(box_1.shape)
plt.imshow(box_1, cmap='viridis', vmin=1000, vmax=12000)
cross_x = box_1.mean(axis=0)
print(cross_x.shape)
x = np.arange(0,box_1.shape[1])

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(9,4))
ax.step(x,cross_x)
ax.set_xlabel('x axis')
ax.set_ylabel('Number of counts')
ax.grid()

In [ ]:
from scipy.optimize import curve_fit
from scipy import asarray as ar,exp
def gaussline(x,cont,a,x0,sigma):
    return cont + a*np.exp(-np.power(x - x0, 2)/(2*np.power(sigma, 2)))

mean = 1000
sigma = 10
pars, cov = curve_fit(f=gaussline, xdata=x, ydata=cross_x, p0=[5000,mean, 20, sigma])


In [ ]:
print(pars)
continuo         = round(pars[0],2)
posicion         = round(pars[2],2)
sigma            = round(pars[3],2)
print('continuo         =',continuo)
print('posición central =',posicion)
print('sigma            =',sigma)

In [ ]:
ajuste = gaussline(x,pars[0],pars[1],pars[2],pars[3])

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(9, 4))
ax.step(x,cross_x,label='original')
ax.step(x,ajuste,label='fitted gaussian')
ax.set_xlabel('x axis [pixels]')
ax.set_ylim(0,20000)
plt.legend()
ax.set_ylabel('Number of counts')
ax.grid()

In [ ]:
# pixel = 15µm, 0.2138 arcsec/pixel
sigma_arcsec = sigma * 0.2138
FWHM = 2.355 * sigma_arcsec
print(sigma,sigma_arcsec,FWHM)

In [ ]:
x = np.subtract(x,posicion)
x_arcsec = np.multiply(x,0.2138)

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(9, 4))
ax.plot(x_arcsec,cross_x,'bo',label='original')
ax.set_xlabel('x axis [arcsec]')
ax.set_ylim(0,20000)
ax.set_xlim(-3,2.5)
plt.legend()
ax.set_xticks(np.arange(-3,4,0.5))
ax.set_ylabel('Number of counts')
ax.grid()